In [1]:
from maml import *
import matplotlib.pyplot as plt

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Current device:", device)

Current device: cuda


In [ ]:
# maml = MAML(5, 0.0001, 0.00001, 0, 2, 1, -1, 1, eqname='poisson', zero_shot=True, load=True, modelpath='models/poisson_zs_2000_ref.data')
maml = MAML(1, 0.01, 0.0001, 1, 2, 1, -1, 1, eqname='burgers', zero_shot=True, load=True, modelpath='models/burgers_zs_1000_ref.data')
# maml = MAML(5, 0.01, 0.0001, 0, 2, 1, low=-1, high=1, eqname='poisson')

train_loss, val_loss, val_ood_loss, nrmse, model = maml.train(1000, 10000, 1)

Initializing MAML-PINN model
Current device:  cuda
Model loaded successfully from models/burgers_zs_1000_ref.data
cuda
Zero shot mode is True
Finished initialization of MAML-PINN model
Start MAML training at iteration 0
[0.003183098861837907]
[0.06366197723675814]
Validation before training Pre-Adaptation (0.003)| Inner_loss_I: 0.0008 | Inner_loss_B: 0.0004 | Inner_loss_F: 0.9546 | Inner_loss: 0.9558 | NRMSE: 0.1140
Validation before training (0.003)| Inner_loss_I: 0.1122 | Inner_loss_B: 0.0136 | Inner_loss_F: 0.0983 | Inner_loss: 0.2241 | NRMSE: 0.9288
validation OOD before training Pre-Adaptation (0.064)| Inner_loss_I: 0.0004 |Inner_loss_B: 0.0003 | Inner_loss_F: 270.4771 | Inner_loss: 270.4778 | NRMSE: 0.3075
validation OOD before training (0.064)| Inner_loss_I: 74.4940 |Inner_loss_B: 61.9307 | Inner_loss_F: 332.2917 | Inner_loss: 468.7164 | NRMSE: 1.0013


In [ ]:
import pandas as pd

nrmse_df = pd.DataFrame(nrmse)
val_loss_df = pd.DataFrame(val_loss)
val_ood_loss_df = pd.DataFrame(val_ood_loss)
train_loss_df = pd.DataFrame(train_loss)

nrmse_df.to_csv('data/nrmse_poisson_zs_maml.csv')
val_loss_df.to_csv('data/val_loss_poisson_zs_maml.csv')
val_ood_loss_df.to_csv('data/val_ood_loss_poisson_zs_maml.csv')
train_loss_df.to_csv('data/train_loss_zs_maml.csv')

In [ ]:
plt.plot(np.array(nrmse['nrmse_val_pre_adapt']))
plt.plot(np.array(nrmse['nrmse_val']))
# plt.plot(np.array(val_loss['inner_loss'])[:, -1])

In [ ]:
model = PINN(20, 5, dim=1, param_num=2, zero_shot=True)
model.load_state_dict(torch.load('models/poisson_zs_5000_ref.data'))
model.to(device)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Current device:", device)
test_x = np.linspace(-10, 10, num=100).reshape(-1, 1)
# test_alpha = np.full((100, 1), alpha[2])
# test_beta = np.full((100, 1), beta[2])
test_alpha = np.full((100, 1), -0.830)
test_beta = np.full((100, 1), -0.617)
test_in = np.hstack((test_x, test_alpha, test_beta))
test_u = model(torch.Tensor(test_in).to(device))
X = test_x

Y = np.sin(test_alpha * X) + np.cos(test_beta * X) + 0.1 * X



In [ ]:
C = test_u.cpu().detach().numpy()
plt.figure(figsize=(10, 8))
plt.scatter(X, C, label='PINN')
plt.scatter(X, Y, label='Answer')
plt.legend()
plt.colorbar()

In [ ]:
np.sqrt( np.sum((C-Y)**2) / np.sum(C**2) )

In [ ]:
# steps = [ 1000 / len(val_loss['inner_loss']) * i for i in range(len(val_loss['inner_loss']))]
# idx = [100 * i for i in range(10)]
# loss = train_loss['inner_loss_f'][::100]
# plt.plot(val_loss['inner_loss'], label='loss')
plt.plot(np.mean(nrmse['nrmse_val'], axis=1), label='Val.')
plt.plot(np.mean(nrmse['nrmse_val_ood'], axis=1), label='Val. OOD')
plt.legend()
plt.title('FO-MAML Val. Poisson')
# plt.plot( loss)
# plt.plot(nrmse['nrmse_val'])
# plt.plot(nrmse['nrmse_val_ood'])
# plt.plot(train_loss['inner_loss'])

In [ ]:
from train import *

In [ ]:
epochs = 500
lr = 0.001

In [ ]:
# loss_i, loss_b, loss_f, loss, model, val_loss, val_ood_loss, nrmse_set = train(epochs=epochs, lr=lr, i_size=100, b_size=100, f_size=1000, zero_shot=False, load=True, load_data='models/maml_burgers_5000.data', alpha_list=0.01 / np.pi, eqname='burgers')
loss_i, loss_b, loss_f, loss, model, val_loss, val_ood_loss, nrmse_set = train(epochs=epochs, lr=lr, i_size=0, b_size=2, f_size=10, zero_shot=False, load=True, load_data='maml.data', alpha_list=0.7, beta_list=-0.9, eqname='poisson')

In [ ]:
# loss_i_2, loss_b_2, loss_f_2, loss_2, model_2, val_loss_2, val_ood_loss_2, nrmse_set_2 = train(epochs=epochs, lr=lr, i_size=100, b_size=100, f_size=1000, zero_shot=False, load=False, load_data='maml.data', alpha_list=0.01 / np.pi, eqname='burgers')
loss_i_2, loss_b_2, loss_f_2, loss_2, model_2, val_loss_2, val_ood_loss_2, nrmse_set_2 = train(epochs=epochs, lr=lr, i_size=0, b_size=2, f_size=10, zero_shot=False, load=False, load_data='maml.data', alpha_list=0.7, beta_list=-0.9, eqname='poisson')

In [ ]:
plt.title('Loss Few shot')
plt.yscale('log')
plt.plot(loss, label='MAML')
plt.plot(loss_2, label='Random')
plt.legend()

In [ ]:
plt.title('Relative error Few shot')
plt.yscale('log')
plt.plot(nrmse_set, label='MAML')
plt.plot(nrmse_set_2, label='Random')
plt.legend()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Current device:", device)
test_x = np.linspace(-1, 1, num=100).reshape(-1, 1)
# test_alpha = np.full((100, 1), alpha[2])
# test_beta = np.full((100, 1), beta[2])
test_alpha = np.full((100, 1), 0.7)
test_beta = np.full((100, 1), 0.7)
# test_in = np.hstack((test_x, test_alpha, test_beta))
test_u = model_2(torch.Tensor(test_x).to(device))
X = test_x

Y = np.sin(test_alpha * X) + np.cos(test_beta * X) + 0.1 * X


In [ ]:
C = test_u.cpu().detach().numpy()
plt.figure(figsize=(10, 8))
plt.scatter(X, C, label='PINN')
plt.scatter(X, Y, label='Answer')
plt.legend()
plt.colorbar()

In [ ]:
plt.plot(loss)
plt.plot(loss_2)

In [ ]:
from burgers import *

In [ ]:
vtn = 101
vxn = 101
nu = 0.01 / np.pi
vx = np.linspace(-1, 1, vxn)
vt = np.linspace(0, 1, vtn)

vu = burgers_viscous_time_exact1(nu, vxn, vx, vtn, vt)

x, t = np.meshgrid(vx, vt)
x = x.reshape(-1, 1)
t = t.reshape(-1, 1)

plt.scatter(x, t, c=vu, cmap='seismic')
plt.colorbar()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Current device:", device)

In [ ]:
vtn = 101
vxn = 101
nu = 0.01 / np.pi
vx = np.linspace(-1, 1, vxn)
vt = np.linspace(0, 1, vtn)
x, t = np.meshgrid(vx, vt)
x = x.reshape(-1, 1)
t = t.reshape(-1, 1)
alpha = np.full((x.shape), nu)
pred = model_2(torch.Tensor(np.hstack((x, t))).to(device)).detach().cpu().numpy()
# pred = model(torch.Tensor(np.hstack((x, t))).to(device)).detach().cpu().numpy()
truth = burgers_viscous_time_exact1(nu, vxn, vx, vtn, vt).T.reshape(-1, 1)

In [ ]:
plt.scatter(x, t, c=pred, cmap='seismic')
plt.colorbar()